In [2]:
from langchain_community.document_loaders import JSONLoader

In [68]:
type(data)

list

In [83]:
loader = JSONLoader(
    file_path='sample.json',
    text_content=False,
    jq_schema='.faqs[]')

data = loader.load()
type(data)

list

In [84]:
for i in range(len(data)):
    print(len(data[i].page_content))

188
194
286
241
272


In [88]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

# splitter = RecursiveJsonSplitter(max_chunk_size=300)
splitter = CharacterTextSplitter(chunk_overlap=0, chunk_size=300)

In [87]:
data

[Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 1}, page_content='{"question": "What is Python?", "tags": ["programming", "python", "basics"], "answer": "Python is a high-level, interpreted programming language known for its simplicity and versatility."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 3}, page_content='{"question": "How do I create a virtual environment in Python?", "tags": ["python", "virtualenv", "environment"], "answer": "You can create a virtual environment in Python by running `python -m venv <env_name>` and activating it using the respective command for your o

In [89]:
texts = splitter.split_documents(data)

In [91]:
data

[Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 1}, page_content='{"question": "What is Python?", "tags": ["programming", "python", "basics"], "answer": "Python is a high-level, interpreted programming language known for its simplicity and versatility."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'),
 Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 3}, page_content='{"question": "How do I create a virtual environment in Python?", "tags": ["python", "virtualenv", "environment"], "answer": "You can create a virtual environment in Python by running `python -m venv <env_name>` and activating it using the respective command for your o

In [98]:
import os
from dotenv import load_dotenv

load_dotenv()
COHERE_API = os.environ.get("COHERE_API")
SUPABASE_SERVICE_KEY=os.environ.get("SUPABASE_SERVICE_KEY")
SUPABASE_URL="https://uggjxkgfewdwgxbdivox.supabase.co"



In [108]:
from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(cohere_api_key=COHERE_API,model="embed-english-v3.0")


In [100]:
from supabase.client import Client, create_client
supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

In [112]:
from langchain_community.vectorstores import SupabaseVectorStore

vector_store = SupabaseVectorStore.from_documents(
    texts,
    embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

In [115]:
query = "What does pydantic do?"
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

print(matched_docs[0])



(Document(metadata={'source': '/Users/sidhantsriv/code/projects/yantra/sample.json', 'seq_num': 2}, page_content='{"question": "What is Pydantic?", "tags": ["python", "validation", "models"], "answer": "Pydantic is a Python library for data validation and settings management using Python type annotations."}'), 0.654289083369411)
